## 1. Requirements

In [6]:
import pandas as pd
import numpy as np 
import datetime as dt
import seaborn as sns
import scipy
import implicit
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## 2. Create connection

In [7]:
print(" {}, Recommendation engine enabled ".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S")))

 19/08/2022 10:38:03, Recommendation engine enabled 


db_connectiion

In [8]:
from sqlalchemy import create_engine

POSTGRES_ADDRESS = 'xxx.xxx.xxx.xxx' 
POSTGRES_PORT = 'xxxxx'
POSTGRES_USERNAME = 'xxxx' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'xxxx'
POSTGRES_DBNAME = 'xxxxxxxx' ## CHANGE THIS TO YOUR DATABASE NAME
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx_wh = create_engine(postgres_str)

In [9]:
x1 = " {}, - Data retrieved from database ".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

In [10]:
df = pd.read_sql_query(''' 
select 

	*

	from transaction	
	
	
	  ;''', cnx_wh)

In [11]:
# query RFM
df_rfm = pd.read_sql_query(''' 
select 
	*

	from customer;''', cnx_wh)

In [12]:
print(" {}, - Data retrieved successfully ".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S")))

 19/08/2022 10:38:25, - Data retrieved successfully 


## 2. Data Preprocessing

In [13]:
x2 = " {}, - Data pre-processing starts.....".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

In [14]:
df['id_product_text'] = df['id_product']

In [15]:
df = df[~df['id_product'].isnull()]

In [16]:
df['tgl_transaksi'] = pd.to_datetime(df['tgl_transaksi']).dt.date
df = df.sort_values(by=['tgl_transaksi'])

In [17]:
df['id_customer'] = df['id_customer'].astype('str')
df['id_customer'] = df['id_customer'].str.strip()

In [18]:
print(" {}, - Data pre-processing succesfully".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S")))

 19/08/2022 10:38:36, - Data pre-processing succesfully


## 3. Collaborative Filtering & Correlation

In [19]:
x3 = " {}, Recommendation 1 is running ".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

== Data Pre-processing ==

In [20]:
df = df[~df['id_product'].isin(['0','77','326','328'])]

In [21]:
df = df[~df['id_customer'].isin(['1307190000295','999999999','9999999999','999999999999','9999999999999'])]

In [22]:
# create_invoice
def invoice_fun(x):
    return str(x['tgl_transaksi']) + str(x['id_customer'])

In [23]:
# add quantity
df['quantity'] = 1

In [24]:
df['invoice_id'] = df[['tgl_transaksi','id_customer']].apply(invoice_fun,axis=1)

== Data == 

In [ ]:
df

In [ ]:
df.groupby(by='id_customer').count().reset_index()

== Descriptive Information ==

In [27]:
df_fav_product = df.groupby(by=['id_customer','id_product']).count().reset_index().sort_values(by='id_transaksi',ascending=False).drop_duplicates(subset=['id_customer'])[[
    'id_customer','id_product'
]]

In [28]:
dff = df_fav_product.rename(columns = {'id_product':'id_product_fav'})

== Modelling ==

In [29]:
df_baskets = df[['invoice_id', 'id_product', 'id_product_text', 'quantity']]

In [ ]:
df.groupby('id_product_text').agg(
    orders=('invoice_id', 'nunique'),
    quantity=('quantity', 'sum')
).sort_values(by='orders', ascending=False).head(10)

In [ ]:
df_items = df_baskets.pivot_table(index='invoice_id', columns=['id_product_text'], values='quantity').fillna(0)
df_items.head(3)

In [32]:
def get_recommendations(df, item):
    
    recommendations = df.corrwith(df[item])
    recommendations.dropna(inplace=True)
    recommendations = pd.DataFrame(recommendations, columns=['correlation']).reset_index()
    recommendations = recommendations.sort_values(by='correlation', ascending=False)
    
    return recommendations

In [ ]:
recommendations = get_recommendations(df_items, '15')
recommendations.head(4)[1:]

In [34]:
recommendations = get_recommendations(df_items, '5')
rcm = recommendations.head(4)[1:]
rcm['id_product_text'].values.tolist()

['109', '37', '311']

In [35]:
id_product = []
id_recommendation = []

for i in df['id_product_text'].drop_duplicates():
    recommendations = get_recommendations(df_items, i)
    rcm = recommendations.head(4)[1:]
    id_recommendation.append((rcm['id_product_text'].values.tolist()))
    id_product.append(i)

In [36]:
df_recommendation = pd.DataFrame({
    'id_product' : id_product,
    'id_recommendation' : id_recommendation
})

In [37]:
dff = dff.merge(df_recommendation,left_on='id_product_fav', right_on='id_product', how='left')

In [38]:
dff = dff.merge(df_rfm,on='id_customer',how='left')

## 4. Recommendation by RFM

In [39]:
x4 = " {}, Recommendation 2 is running ".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

In [40]:
dfr = dff.copy()

In [41]:
dfs = dfr[['segment']].drop_duplicates()

In [42]:
segment = []
id_product_fav_rfm = []

for i in dfs['segment']:
    x = dfr[dfr['segment'] == i][['id_product_fav']].value_counts().reset_index()[['id_product_fav']][:2]
    id_product_fav_rfm.append(x['id_product_fav'].values.tolist())
    segment.append(i)
    

In [43]:
dfrfm= pd.DataFrame({
    'segment': segment,
    'id_recommendation_rfm': id_product_fav_rfm
})

In [ ]:
dfrfm

In [45]:
df_result = dff.merge(dfrfm,on='segment',how='left')

== insert to database == 

In [46]:
x5 = " {}, - Data sent to data warehouse,please wait.....".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

In [47]:
from sqlalchemy import create_engine

POSTGRES_ADDRESS = 'xxx.xxx.xxx.xxx' 
POSTGRES_PORT = 'xxxxx'
POSTGRES_USERNAME = 'xxxx' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'xxxx'
POSTGRES_DBNAME = 'xxxxxxxx' ## CHANGE THIS TO YOUR DATABASE NAME
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx_wh = create_engine(postgres_str)

In [48]:
df_result.to_sql('confidential_recommendation_engine', con=conn, if_exists='replace',index=False, method='multi')

In [49]:
print(" - {}, Recommendation engine disabled ".format(dt.datetime.now().strftime("%d/%m/%Y %H:%M:%S")))

 - 19/08/2022 10:42:27, Recommendation engine disabled 
